# Задание 1


Выберите 5 языков в википедии (не тех, что использовались в семинаре). Скачайте по 10 случайных статей для каждого языка. Предобработайте тексты, удаляя лишние теги/отступы/разделители (если они есть). Разделите тексты на предложения и создайте датасет, в котором каждому предложению соответствует язык. Кластеризуйте тексты, используя эбмединг модель из прошлого семинара и любой алгоритм кластеризации. Проверьте качество кластеризации с помощь метрики ARI. Отдельно проанализируйте 3 ошибочно кластеризованных текста (если такие есть).

In [19]:
import wikipedia
import numpy as np

In [20]:
def get_random_page():
    try:
        return wikipedia.random(1)
    except Exception:
        return get_random_page()
    
def get_page_by_name(name:str):
    try:
        return wikipedia.page(name)
    except wikipedia.DisambiguationError as e:
        random_option = np.random.choice(e.options)
        return get_page_by_name(random_option)


####  Код ниже можнно не запускать, под ним ячейка со спиклованным объектом

In [29]:
lang_to_pages = {
    "zh": dict(),  #  китайский
    "ja": dict(),  #  японский
    "ko": dict(),  #  корейский
    "hi": dict(),  #  хинди
    "lv": dict(),  #  латышский
}
for lang in lang_to_pages:
    wikipedia.set_lang(lang)
    for _ in range(10):
        random_page = get_random_page()
        lang_to_pages[lang][random_page] = get_page_by_name(random_page).content

d:\HSE subj data\актуальные проблемы компьютерной лингвистики\.venv\Lib\site-packages\wikipedia\wikipedia.py:389: GuessedAtParserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("html.parser"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 389 of the file d:\HSE subj data\актуальные проблемы компьютерной лингвистики\.venv\Lib\site-packages\wikipedia\wikipedia.py. To get rid of this warning, pass the additional argument 'features="html.parser"' to the BeautifulSoup constructor.

  lis = BeautifulSoup(html).find_all('li')


In [2]:
import pickle

In [34]:
with open(r"data/lang_to_pages.pkl", "wb") as file:
    pickle.dump(lang_to_pages, file)

## Загрузка объекта

In [2]:
import pickle

with open(r"data/lang_to_pages_COPY.pkl", "rb") as file:
    lang_to_pages = pickle.load(file)


### Чистка текстов

In [3]:
import re



class NormRule:
    def __init__(self, from_:str, to_:str) -> None:
        self.from_ = from_
        self.to_ = to_

    def apply_rule(self, text:str) -> str:
        return text.replace(self.from_, self.to_)


class RegNormRule(NormRule):

    def apply_rule(self, text: str) -> str:
        return re.sub(self.from_, self.to_, text)



class Normalizer:

    rules = (
        NormRule("==", ""),
        RegNormRule(r"\s+", " "),
        RegNormRule("・+", " "),
    )


    @classmethod
    def normalize(cls, text:str) -> str:
        for rule in cls.rules:
            text = rule.apply_rule(text)
        return text


In [4]:
for lang in lang_to_pages:
    lang_to_title = lang_to_pages[lang]
    for title in lang_to_title:
        lang_to_title[title] = Normalizer.normalize(lang_to_title[title])

In [5]:
import nltk
from nltk import sent_tokenize
nltk.download('punkt_tab')


[nltk_data] Downloading package punkt_tab to
[nltk_data]     /home/smertlove/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [6]:
from random import choice

In [7]:
key = choice(list(lang_to_pages["ja"].keys()))
txt = lang_to_pages["ja"][key] #  ko hi lv

print(repr(txt))
print(len(sent_tokenize(txt, "japanese")))


'HED.Cycling Products(日:ヘッド サイクリング プロダクト)は、ミネソタ州セントポールの自転車店Grand Performanceオーナーであったスティーブ ヘッド(英:Steve Hed)により、「トライアスリートの為に現代的で手頃な価格のエアロ ディスクホイールを作る」というコンセプトの元、1984年に誕生した自転車部品メーカーである。 ロードレースの世界ではあまり目立たないメーカーだが、トライアスリートやメッセンジャーなどシングルスピードバイクのユーザーからは絶大な人気のあるホイールブランドである。 歴史 1984年：ミネソタ州セントポールの自転車店オーナーであったスティーブ ヘッドにより創業 1989年：当時、模型飛行機の製作に時間を費やし、飛行のあらゆる側面を研究していたスティーブは、従来の自転車のホイールはV字型のくさび形をしていて、空気力学上で、アドバンテージが得られていない事を発見し、リム断面がトロイダル型のHED CXを開発、60mmハイト、26mm幅という画期的なリムを設計 1988年：スティーブはトライアスロンの選手に空気力学的な優位性を与えるために、ウォーターボトルの収納場所をライダーの後ろ、そしてライダーが生み出す自然なスリップストリームの内側に再配置することを思いつきました。この新製品を「Tails」と名付け、シート後方用ボトルホルダーを開発。 1992年：スティーブ達は、初のカーボンファイバー製トロイダル型ホイールを発表しました。「ジェットスタイル ホイール」と名付け、業界をリードするジェットシリーズの初製品となりました。 1995年：アメリカのマウンテンバイクブームが起こると、HEDはエアロダイナミックの専門知識をゲレンデやパークに持ち込こみ、エアロダイナミックのダウンヒルMTBとBMXホイールを発表。 1998年：HED社はアメリカのデュポン社と協力して、従来のスポークの代わりに3本の空力フォイルを備えたカーボンホイールという、型破りなホイールデザインを獲得しました、3本のスポークは丸みを帯びたリーディングエッジとシャープなトレーリングエッジを持ち、これまでにないパフォーマンスの優位性を生み出しました。 2001年：更なる製品開発を求めていたスティーブは、今度はエアロバーに目を向け、１0年以上かけてHEDサイク

LookupError: 
**********************************************************************
  Resource [93mpunkt_tab[0m not found.
  Please use the NLTK Downloader to obtain the resource:

  [31m>>> import nltk
  >>> nltk.download('punkt_tab')
  [0m
  For more information see: https://www.nltk.org/data.html

  Attempted to load [93mtokenizers/punkt_tab/japanese/[0m

  Searched in:
    - '/home/smertlove/nltk_data'
    - '/home/smertlove/sandbox/hse/nlp_hw/venv/nltk_data'
    - '/home/smertlove/sandbox/hse/nlp_hw/venv/share/nltk_data'
    - '/home/smertlove/sandbox/hse/nlp_hw/venv/lib/nltk_data'
    - '/usr/share/nltk_data'
    - '/usr/local/share/nltk_data'
    - '/usr/lib/nltk_data'
    - '/usr/local/lib/nltk_data'
**********************************************************************


### NLTK не поддерживает такую экзотику, значит будем городить своё.

In [8]:
class Tokenizer:

    delimiter = "."

    @classmethod
    def tokenize(cls, text:str) -> list[str]: 
        return text.split(cls.delimiter)


class HieroglyphicTokenizer(Tokenizer):
    delimiter = "。"


class DevanagariTokenizer(Tokenizer):
    delimiter = "।"


class RegularTokenizer(Tokenizer):

    @classmethod
    def tokenize(cls, text: str) -> list[str]:
        return nltk.sent_tokenize(text)


lang_to_tokenizer : dict[str, Tokenizer]= {
    "zh": HieroglyphicTokenizer,
    "ja": HieroglyphicTokenizer,
    "ko": Tokenizer,
    "hi": DevanagariTokenizer,
    "lv": RegularTokenizer,
}

In [16]:
corpus = []

for i, lang in enumerate(lang_to_pages):
    tokenizer = lang_to_tokenizer[lang]
    lang_to_title = lang_to_pages[lang]

    for title in lang_to_title:
        sents = tokenizer.tokenize(lang_to_title[title])
        
        for sent in sents:
            corpus.append((sent.strip(), lang, i + 1))


In [17]:
import pandas as pd

In [19]:
df = pd.DataFrame(corpus, columns=("sentence", "language", "lang_id"))

In [20]:
df

,sentence,language,lang_id
0,布拉迪斯拉發施洛雲體育會（斯洛伐克語：ŠK Slovan Bratislava），簡稱施洛雲...,zh,1
1,斯拉夫人是斯洛伐克最為成功的足球俱樂部之一,zh,1
2,在捷克斯洛伐克時代，斯拉夫人是獲得國內聯賽冠軍次數最多的斯洛伐克球隊（共計 8 次）,zh,1
3,球隊還在1969年5月21日以 3–2 击败巴塞罗那，獲得了歐洲盃賽冠軍盃冠军,zh,1
4,斯洛伐克独立后，斯拉夫人成为国内最大豪门，屡次夺得联赛和杯赛冠军,zh,1
...,...,...,...
792,Pilsēta pirmoreiz minēta rakstos 960. gadā.,lv,5
793,"Burgrāfistes centrs, līdz 1460. gadā, tas tika...",lv,5
794,Atsauces Ārējās saites Vikikrātuvē par šo tēmu...,lv,5
795,Skatīt: Dona.,lv,5


###  Загружаем модель

In [12]:
from sentence_transformers import SentenceTransformer

/home/smertlove/sandbox/hse/nlp_hw/venv/lib/python3.10/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange


In [13]:
model = SentenceTransformer('sentence-transformers/all-mpnet-base-v2')

/home/smertlove/sandbox/hse/nlp_hw/venv/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [14]:
from sklearn.cluster import KMeans
from sklearn.metrics import adjusted_rand_score
import numpy as np

###  Получим кластеры:

In [41]:
sents = df['sentence'].values
langs = df["language"].values

X = np.zeros((len(sents), 768))

for i, sent, lang in zip(range(len(sents)), sents, langs):
    X[i] = model.encode(sent)

cluster = KMeans(
    n_clusters = len(lang_to_pages),
    max_iter = 500,
    algorithm = 'elkan',
    random_state=21  ##  нужно чтобы результат кластеризации всегда был одинаковый
    )
cluster.fit(X)

predicted_labels = np.array(cluster.labels_)+1
true_labels = df["lang_id"]

###  Посчитаем ARI:

In [42]:
ARI = adjusted_rand_score(true_labels, predicted_labels)
        
print(np.mean(ARI)) # усредненная метрика

0.5025935809760352


###  Теперь надо собоставить кластеры языкам.

In [43]:
#  Посмотрим, как распределились кластеры:
predicted_labels

array([5, 3, 3, 4, 3, 3, 3, 3, 3, 3, 3, 3, 1, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 1, 1, 1, 3, 1, 3, 1, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,

### Кластеры явно видны.

Это значит, что в целом мы можем сопоставить язык и номер кластера. Сделаем это так:

In [50]:
#  Посмотрим, сколько у нас предложений на каждый язык:

grouped_df = df.groupby("language")

langs_and_counts = [
        (key, len(grouped_df.get_group(key)))
        for key
        in ("zh", "ja", "ko", "hi", "lv")  ##  порядок важен
    ]
langs_and_counts


[('zh', 231), ('ja', 201), ('ko', 229), ('hi', 45), ('lv', 91)]

In [62]:
#  Посчитаем сымые частнотные предсказания для каждого языка

def get_most_common_from_list(target:list, keys:set) -> int:

    counts = {obj : target.count(obj) for obj in keys}
    return max(counts, key=lambda c: counts[c])


cluster_to_lang = dict()

clusters = {1,2,3,4,5}

offset = 0
for lang, count in langs_and_counts:
    window = predicted_labels[offset:offset+count]
    most_common = get_most_common_from_list(list(window), clusters)

    cluster_to_lang[most_common] = lang

    clusters.remove(most_common)
    offset += count

print(cluster_to_lang)

{3: 'zh', 1: 'ja', 2: 'ko', 5: 'hi', 4: 'lv'}


In [65]:
class Predictor:
    @classmethod
    def cast_vector(cls, row):
        return np.array(list(map(lambda x: x.astype('double'), row)))


    @classmethod
    def predict(cls, text:str) -> str:
        emb = model.encode(text).reshape(1, -1)
        emb = cls.cast_vector(emb)

        return cluster_to_lang[int(cluster.predict(emb)[0])+ 1] 

In [82]:
sample_texts = (
    "红红的小脸儿温暖我的心窝",  ##  zh
    "点亮我生命的火 火火火火",   ##  zh
    "一步一步地会到目的",          ##  zh
    "蝸牛そろそろ登れ富士の山",  ##  ja
    "私の名は吉良吉影。",   ##  ja
    "오빤 강남 스타일, uh 강남 스타일 옵, 옵-옵-옵, 오빤 강남 스타일, uh 강남 스타일 옵, 옵-옵-옵, 오빤 강남 스타일",  ##  ko
    "내가 무슨 언어야?",  ##  ko
    "잘 지내세요? ",  ##  ko
    "हिन्दी में टाइप करें",  ##  hi
    "खुशी उन लोगों को नहीं आएगी जो नहीं जानते कि उनके पास पहले से क्या सराहना है",  ##  hi
    "Jo es gauži bēdājos, Jo nelaime priecājas",  ##  lv
    "Kad cer, tad notic tik viegli. 1 Patīk. – Ērihs Marija Remarks. ", ##  lv

)

for text in sample_texts:
    print(text, Predictor.predict(text), sep="\t")

红红的小脸儿温暖我的心窝	zh
点亮我生命的火 火火火火	zh
 一步一步地会到目的	zh
蝸牛そろそろ登れ富士の山	zh
私の名は吉良吉影。	zh
오빤 강남 스타일, uh 강남 스타일 옵, 옵-옵-옵, 오빤 강남 스타일, uh 강남 스타일 옵, 옵-옵-옵, 오빤 강남 스타일	ko
내가 무슨 언어야?	ko
잘 지내세요? 	ko
हिन्दी में टाइप करें	hi
खुशी उन लोगों को नहीं आएगी जो नहीं जानते कि उनके पास पहले से क्या सराहना है	hi
Jo es gauži bēdājos, Jo nelaime priecājas	hi
Kad cer, tad notic tik viegli. 1 Patīk. – Ērihs Marija Remarks. 	hi


###  Путает пары японский-китайский и хинди-латышский.

In [86]:
from sklearn.metrics import classification_report, confusion_matrix

In [87]:
true_labels = []
predicted_labels = []

for i, sent, lang in zip(range(len(sents)), sents, langs):
    true_labels.append(lang)
    predicted_labels.append(Predictor.predict(sent))

In [88]:
print(classification_report(true_labels, predicted_labels))
print(confusion_matrix(true_labels, predicted_labels))

              precision    recall  f1-score   support

          hi       0.32      0.84      0.46        45
          ja       0.07      0.01      0.02       201
          ko       0.99      0.60      0.75       229
          lv       0.00      0.00      0.00        91
          zh       0.52      0.97      0.68       231

    accuracy                           0.50       797
   macro avg       0.38      0.48      0.38       797
weighted avg       0.47      0.50      0.44       797

[[ 38   7   0   0   0]
 [  0   2   1   0 198]
 [  0   4 138  82   5]
 [ 80  11   0   0   0]
 [  1   6   0   1 223]]


# Задание 2

Загрузите корпус `annot.opcorpora.no_ambig_strict.xml.bz2` с OpenCorpora. Найдите в корпусе самые частотные морфологически омонимичные словоформы (те, которым соответствует разный грамматический разбор в разных предложениях). Также найдите словоформы с самых большим количеством вариантов грамматических разборов.

In [89]:
import lxml

In [146]:

with open(r"data/annot.opcorpora.no_ambig_strict.xml", "rb") as file:
    open_corpora = lxml.etree.fromstring(file.read())

In [121]:
corpus = []


for sentence in open_corpora.xpath('//tokens'):
    sent_tagged = []
    for token in sentence.xpath('token'):
        word = token.xpath('@text')
        gram_info = token.xpath('tfr/v/l/g/@v')
        sent_tagged.append([word[0].lower()] + gram_info)
    
    corpus.append(sent_tagged)

In [122]:
corpus[1]

[['сохранится', 'VERB', 'perf', 'intr', 'sing', '3per', 'futr', 'indc'],
 ['ли', 'PRCL'],
 ['градус', 'NOUN', 'inan', 'masc', 'sing', 'nomn'],
 ['дискуссии', 'NOUN', 'inan', 'femn', 'sing', 'gent'],
 ['в', 'PREP'],
 ['новом', 'ADJF', 'Qual', 'masc', 'sing', 'loct'],
 ['сезоне', 'NOUN', 'inan', 'masc', 'sing', 'loct'],
 ['?', 'PNCT']]

In [151]:
from copy import deepcopy

In [152]:
d:dict[str, set] = dict()

entry_tmpl = {
    "count": 0,
    "tagset_set": set()
}

for sentence in corpus:
    for token in sentence:
        word = token[0]
        tagset = ';'.join(token[1:])

        entry = d.get(word, deepcopy(entry_tmpl))
        entry["tagset_set"].add(tagset)
        entry["count"] += 1

        d[word] = entry

In [161]:
d_ambig = {
    key:d[key]
    for key
    in d
    if len(d[key]["tagset_set"]) > 1
}

In [166]:
from collections import Counter

counter_manytagsets = Counter({
    key: len(d_ambig[key]["tagset_set"])
    for key
    in d_ambig
})


counter_manyforms   = Counter({
    key: d_ambig[key]["count"]
    for key
    in d_ambig
})

print("Cамые частотные морфологически омонимичные словоформы:")
print("слово", "количество форм", "статья", sep="\t")
for word, count in counter_manyforms.most_common(10):
    print(word, count, d[word], sep="\t")

print()

print("Cловоформы с самым большим количеством вариантов грамматических разборов:")
print("слово", "количество тегсетов", "статья", sep="\t")
for word, count in counter_manytagsets.most_common(10):
    print(word, count, d[word], sep="\t")

Cамые частотные морфологически омонимичные словоформы:
слово	количество форм	статья
в	2059	{'count': 2059, 'tagset_set': {'PREP', 'NOUN;inan;masc;Fixd;Abbr;sing;gent'}}
на	786	{'count': 786, 'tagset_set': {'PREP', 'PRCL'}}
с	613	{'count': 613, 'tagset_set': {'PREP', 'PRCL'}}
и	574	{'count': 574, 'tagset_set': {'CONJ', 'PRCL'}}
о	213	{'count': 213, 'tagset_set': {'PREP', 'INTJ'}}
году	115	{'count': 115, 'tagset_set': {'NOUN;inan;masc;sing;loc2', 'NOUN;inan;masc;sing;datv'}}
а	113	{'count': 113, 'tagset_set': {'CONJ', 'INTJ'}}
этом	104	{'count': 104, 'tagset_set': {'ADJF;Subx;Apro;Anph;masc;sing;loct', 'NPRO;neut;sing;loct', 'ADJF;Subx;Apro;Anph;neut;sing;loct'}}
россии	91	{'count': 91, 'tagset_set': {'NOUN;inan;femn;Sgtm;Geox;sing;gent', 'NOUN;inan;femn;Sgtm;Geox;sing;datv', 'NOUN;inan;femn;Sgtm;Geox;sing;loct'}}
было	89	{'count': 89, 'tagset_set': {'VERB;impf;intr;neut;sing;past;indc', 'PRCL'}}

Cловоформы с самым большим количеством вариантов грамматических разборов:
слово	количество 

## Задание 3
Загрузите один и з файлов корпуса Syntagrus - https://github.com/UniversalDependencies/UD_Russian-SynTagRus/tree/master (можно взять тестовый)

Преобразуйте все разборы предложений в графовые структуры через DependencyGraph, выберите 3 любых отношения и для каждого найдите топ-5 самых встречаемых пар слов, связанных этим отношением. 

Для самой частотной пары слов в каждом из отношений вытащите все подзависимые слова для каждого из них во всех предложениях (используя `flatten(get_subtree(d.nodes, index_of_a_word)` и сортируя результат по порядку слов в предложениях, аналогично тому как я делал с summaries только у вас будет два слова) 
В итоге у вас должен получится что-то такое:

```
### отношение
relation_name

### топ 5 пар слов связанных этим отношением
(word1, word2), (word3, word4), (word5, word6), (word7, word8), (word9, word10)

### подзависимые для самого частотного
(subword word1 subword, word2 subword subword)

... (и так три раза)
```


In [36]:
with open(r"data/ru_syntagrus-ud-test.conllu", "r", encoding="utf-8") as file:
    data = [sentence.split("\n") for sentence in file.read().split('\n\n')]
    data = tuple(map(
        lambda sentence: "\n".join([tok for tok in sentence if not tok.startswith("#")]),
        data
    ))

In [51]:
def flatten(l):
    flat = []
    for el in l:
        if not isinstance(el, list):
            flat.append(el)
        else:
            flat += flatten(el)
    return flat

def get_subtree(nodes, node):
    
    
    if not nodes[node]['deps']:
        return [node]
    
    else:
        return [node] + [get_subtree(nodes, dep) for rel in nodes[node]['deps'] 
                         if rel != 'punct'  # пунктуацию доставать не будем
                         for dep in nodes[node]['deps'][rel]]

In [32]:
from nltk.parse import DependencyGraph
from collections import Counter
import warnings
import os
warnings.filterwarnings('ignore')

In [45]:
corpus = []
for sentence in data:
    d = DependencyGraph(sentence, cell_separator="\t")
    corpus.append(d)

###  Отношение NOUN + amod

In [87]:

noun_to_amod_forms : dict[str, dict[str, int]] = dict()
# noun_to_amod_lemmas: dict[str, dict[str, int]] = dict()


for d in corpus:
    for node_i, node in d.nodes.items():
        if node['ctag'] == 'NOUN' and 'amod' in node['deps']:

            noun = node
            attrs = [d.nodes[i] for i in node['deps']['amod']]

            for attr in attrs:
                phrase_form  = f'{attr["word"]} {noun["word"]}'
                noun_to_amod_forms[phrase_form] = noun_to_amod_forms.get(phrase_form, 0) + 1

                # phrase_lemma = f'{attr["lemma"]} {noun["lemma"]}'
                # noun_to_amod_lemmas[phrase_lemma] = noun_to_amod_lemmas.get(phrase_lemma, 0) + 1


In [90]:
counter = Counter(noun_to_amod_forms)
counter.most_common(5)

[('самом деле', 18),
 ('последнее время', 14),
 ('другой стороны', 14),
 ('1933 года', 13),
 ('пенсионного возраста', 12)]

In [112]:
amod, noun = counter.most_common(1)[0][0].split(" ")
flatten_trees = list()


for d in corpus:
    for node_i, node in d.nodes.items():
        ##  Идея в том, что поддерево amod -- это поддерево noun
        # if node['word'] in (amod, noun):
        if node['word'] == noun:
            flatten_trees.append([d.nodes[i]["word"] for i in sorted(flatten(get_subtree(d.nodes, node_i)))])

In [113]:
for c in flatten_trees[0]:
    print(c)
for c in flatten_trees[1]:
    print(c)

в
Измаильском
деле
Повсюду
был
он
Платов
присутственен
и
подавал
пример
храбрости
На
самом
деле


###  Отношение VERB + advmod

In [134]:

verb_to_amod_forms : dict[str, dict[str, int]] = dict()

for d in corpus:
    for node_i, node in d.nodes.items():
        if node['ctag'] == 'VERB' and 'advmod' in node['deps']:

            verb = node
            attrs = [d.nodes[i] for i in node['deps']['advmod'] if d.nodes[i]['ctag'] in ('ADV', "ADJ")]

            for attr in attrs:
                phrase_form  = f'{attr["lemma"]} {verb["lemma"]}'
                verb_to_amod_forms[phrase_form] = verb_to_amod_forms.get(phrase_form, 0) + 1


counter = Counter(verb_to_amod_forms)

In [135]:
counter.most_common(5)

[('так называть', 32),
 ('так делать', 8),
 ('там быть', 8),
 ('быстро расти', 8),
 ('здесь быть', 6)]

In [142]:
advmod, verb = counter.most_common(1)[0][0].split(" ")
flatten_trees_verb = list()
flatten_trees_advmod = list()


for d in corpus:
    for node_i, node in d.nodes.items():

        if node['lemma'] == verb:
            flatten_trees_verb.append([d.nodes[i]["word"] for i in sorted(flatten(get_subtree(d.nodes, node_i)))])
        elif node['lemma'] == advmod:
            flatten_trees_advmod.append([d.nodes[i]["word"] for i in sorted(flatten(get_subtree(d.nodes, node_i)))])



In [145]:
flatten_trees_verb[3]

['имен', 'своих', 'не', 'называть']

In [149]:
flatten_trees_advmod[6]

['не', 'так', 'как', 'из', 'зрительного', 'зала']

### Отношение ROOT + any

In [166]:

root_to_next_forms : dict[str, dict[str, int]] = dict()

for d in corpus:
    for node_i, node in d.nodes.items():
        if node['ctag'] == 'TOP':

            root = node
            attrs = [d.nodes[i] for i in node['deps']["root"]]

            for attr in attrs:
                phrase_form  = f'{attr["lemma"]}'
                root_to_next_forms[phrase_form] = root_to_next_forms.get(phrase_form, 0) + 1


counter = Counter(root_to_next_forms)

In [167]:
counter.most_common(5)

[('мочь', 197), ('можно', 127), ('быть', 126), ('стать', 103), ('должен', 90)]

In [168]:
root = counter.most_common(1)[0][0]
flatten_trees = list()


for d in corpus:
    for node_i, node in d.nodes.items():
        if node['lemma'] == root:
            flatten_trees.append([d.nodes[i]["word"] for i in sorted(flatten(get_subtree(d.nodes, node_i)))])

In [169]:
flatten_trees[1]

['Он', 'не', 'мог', 'уйти', 'обратно', 'с', 'полным', 'подносом']